In [1]:
import random
import numpy as np
import torch
from transformers import set_seed

In [2]:
seed = random.randrange(2**32)
print(f"🔢 Using random seed: {seed}")

# Seed all RNGs
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)  # also seeds Hugging Face’s Trainer internals

🔢 Using random seed: 370370239


In [3]:
# Cell 1: Install dependencies (don’t upgrade CUDA‑linked packages)
!pip install transformers datasets evaluate box

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.4/162.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.6 MB/s eta 0:00:00
  Created wheel for timeago: filename=timeago-1.0.14-py3-none-any.whl size=26438 sha256=039bce5a9f97c496b8bfd5fac767b466b8df63b6095fa4bdaff0236633825276
  Stored in directory: /root/.cache/pip/wheels/52/98/98/95c0e2ffff3b879aba7805790f9c321

In [8]:
# Cell 2: Imports
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import numpy as np

In [5]:
# Cell 3: Config / Args
args = {
    "model_name_or_path":      "distilbert-base-uncased",
    "output_dir":              "./output/squad",
    "max_seq_length":          384,
    "doc_stride":              128,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size":  16,
    "learning_rate":           3e-5,
    "num_train_epochs":        3,
    "logging_steps":           500,
    "save_steps":              500,
    "seed":                    seed,
    "version_2_with_negative": False,  # set True if you switch to SQuAD v2
}


In [6]:
# Cell 4: Prepare the SQuAD dataset
raw_datasets = load_dataset("squad")                # v1.1
metric       = evaluate.load("squad")

tokenizer = AutoTokenizer.from_pretrained(
    args["model_name_or_path"], use_fast=True
)

column_names = raw_datasets["train"].column_names
question_col = "question"
context_col  = "context"
answer_col   = "answers"
pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):
    tokenized = tokenizer(
        examples[question_col if pad_on_right else context_col],
        examples[context_col  if pad_on_right else question_col],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=args["max_seq_length"],
        stride=args["doc_stride"],
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # Map each tokenized example back to its original example
    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")

    starts, ends = [], []
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized.sequence_ids(i)
        sample_idx = sample_mapping[i]
        answers = examples[answer_col][sample_idx]
        if len(answers["answer_start"]) == 0:
            starts.append(cls_index)
            ends.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char   = start_char + len(answers["text"][0])
            # Find token start/end
            token_start = 0
            while sequence_ids[token_start] != (1 if pad_on_right else 0):
                token_start += 1
            token_end = len(input_ids) - 1
            while sequence_ids[token_end] != (1 if pad_on_right else 0):
                token_end -= 1
            # If answer out of span
            if not (offsets[token_start][0] <= start_char and offsets[token_end][1] >= end_char):
                starts.append(cls_index)
                ends.append(cls_index)
            else:
                while token_start < len(offsets) and offsets[token_start][0] <= start_char:
                    token_start += 1
                starts.append(token_start - 1)
                while offsets[token_end][1] >= end_char:
                    token_end -= 1
                ends.append(token_end + 1)
    tokenized["start_positions"] = starts
    tokenized["end_positions"]   = ends
    return tokenized

# Tokenize train & validation
train_dataset = raw_datasets["train"].map(
    prepare_train_features, batched=True, remove_columns=column_names
)
eval_dataset  = raw_datasets["validation"].map(
    prepare_train_features, batched=True, remove_columns=column_names
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [10]:
# Cell 5: Model, Data Collator, and Trainer Setup
model = AutoModelForQuestionAnswering.from_pretrained(
    args["model_name_or_path"]
)
data_collator = DataCollatorWithPadding(tokenizer)

training_args = TrainingArguments(
    output_dir=args["output_dir"],
    seed=args["seed"],
    per_device_train_batch_size=args["per_device_train_batch_size"],
    per_device_eval_batch_size=args["per_device_eval_batch_size"],
    learning_rate=args["learning_rate"],
    num_train_epochs=args["num_train_epochs"],
    logging_steps=args["logging_steps"],
    save_steps=args["save_steps"],
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # QA uses eval_loss
    overwrite_output_dir=True,
)

def compute_qa_metrics(p):
    start_logits, end_logits = p
    # Use HF helper to align predictions to original contexts
    from transformers import default_data_collator
    # Convert raw logits & features back to answers
    # (you can reuse HF’s postprocess_qa_predictions util if you import it)
    return metric.compute(predictions=preds, references=references)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_qa_metrics,   # optional for span metrics
)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-10-cd8e21d19819>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Cell 6: Train & Evaluate
trainer.train()
metrics = trainer.evaluate()
print(metrics)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sunnysolomon8880 (sunnysolomon8880-cornell-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
